In [1]:
from osgeo import gdal, ogr, osr
import os

In [2]:
# Paths to input and output folders
input_folder = 'input_folder_path'
output_folder = 'output_folder_path'

# Path to the shapefile containing the clipping polygon
shapefile = 'path_shapefile'

In [3]:
# Create a spatial reference object for CRS 4326
crs_4326 = osr.SpatialReference()
crs_4326.ImportFromEPSG(4326)


c:\Users\elima\anaconda3\envs\pygdal\Lib\site-packages\osgeo\osr.py:385: FutureWarning: Neither osr.UseExceptions() nor osr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


0

In [4]:
# Open the shapefile and get the layer
shape_ds = ogr.Open(shapefile)
layer = shape_ds.GetLayer()

In [5]:
# Loop through each GeoTIFF file in the input folder
for input_geotiff in os.listdir(input_folder):
    if input_geotiff.endswith('.tif'):
        input_geotiff_path = os.path.join(input_folder, input_geotiff)
        
        # Open the input GeoTIFF dataset
        input_ds = gdal.Open(input_geotiff_path)

        # Check if input GeoTIFF is not in CRS 4326, and reproject if needed
        if input_ds.GetProjection() != crs_4326.ExportToWkt():
            input_ds = gdal.Warp('reprojected.tif', input_ds, dstSRS=crs_4326.ExportToWkt())
            input_geotiff_path = 'reprojected.tif'  # Update input file path

        # Get the geometry of the clipping feature
        feature = layer.GetFeature(0)  # Assuming you want to clip using the first feature
        clipping_geometry = feature.GetGeometryRef()

        # Create WarpOptions for clipping
        warp_options = gdal.WarpOptions(cutlineDSName=shapefile, cutlineWhere=f'FID={feature.GetFID()}', cropToCutline=True, dstSRS=crs_4326.ExportToWkt())

        # Create the output file path
        output_geotiff_path = os.path.join(output_folder, input_geotiff)

        # Perform the clipping using Warp
        output_ds = gdal.Warp(output_geotiff_path, [input_ds], options=warp_options)

        # Close the datasets
        input_ds = None
        output_ds = None

# Close the shapefile dataset
shape_ds = None